In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter
import seaborn as sns
import datetime as dt
import time
from datetime import date, timedelta
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
from scipy.stats import poisson
import scipy.stats as stats
%matplotlib inline

from google.oauth2.service_account import Credentials
import google_auth_httplib2
import pygsheets
import googleapiclient
from googleapiclient  import discovery
from httplib2 import Http
from oauth2client import file, client, tools
from gspread_formatting.dataframe import format_with_dataframe, BasicFormatter
from gspread_formatting import Color
import logging

from oauth2client.service_account import ServiceAccountCredentials
from google.oauth2 import service_account
import gslides
from gslides import (Frame,Presentation,Spreadsheet,Table,Series,Chart)

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore")

#import sqlalchemy as db
from sqlalchemy import create_engine
import mysql.connector
import psycopg2

#pip install google-api-core==1.32.0
#pip install --upgrade firebase-admin

In [2]:
import firebase_admin
from firebase_admin import credentials, firestore
#pip install google-api-core==1.32.0
#pip install protobuf==3.15.5
#pip install google-cloud-firestore==2.3.4


###>>>>>>>>>>>>>>>>>>>>>>>>>> Firestore credentials
try:
    app = firebase_admin.get_app()
except ValueError as e:
    cred = credentials.Certificate("C:/Users/leemn/Documents/Loop Service Account/accountkey_firebase.json")
    firebase_admin.initialize_app(cred)
db = firestore.client() 

## Client Users

In [17]:
#collect single doc
doc_file = db.collection(u'client-users').document(u'0Gbd3UyDgGMsYKL2nj2H')
my_dict = doc_file.get().to_dict()
print(my_dict)

{'password': '$2a$10$LH3Bvph5uBIzX6w.faGHouq7jqMiq0EfKj1zCFG7vUATG.SpxKcyG', 'created_at': DatetimeWithNanoseconds(2021, 7, 6, 10, 15, 9, 947000, tzinfo=<UTC>), 'mobile_no': '+27000000142', 'lastname': 'Singh', 'firstname': 'Vimal', 'email': 'vimals@nandocas.com', 'created_by': <google.cloud.firestore_v1.document.DocumentReference object at 0x000002E08D010D60>}


In [4]:
# single doc
df = pd.DataFrame()
df = df.append(my_dict, ignore_index=True)
df.head()

NameError: name 'my_dict' is not defined

In [18]:
# collect multiple docs
docs_dict = {}
docs = db.collection(u'client-users').stream()

for doc in docs:
    print(f'{doc.id}: {doc.to_dict()}')
    docs_dict[doc.id] = doc.to_dict()
return docs_dict

0Gbd3UyDgGMsYKL2nj2H: {'created_by': <google.cloud.firestore_v1.document.DocumentReference object at 0x000002E08D010D30>, 'email': 'vimals@nandocas.com', 'created_at': DatetimeWithNanoseconds(2021, 7, 6, 10, 15, 9, 947000, tzinfo=<UTC>), 'firstname': 'Vimal', 'password': '$2a$10$LH3Bvph5uBIzX6w.faGHouq7jqMiq0EfKj1zCFG7vUATG.SpxKcyG', 'lastname': 'Singh', 'mobile_no': '+27000000142'}
0IoJYDUaycxi0e5Fmozg: {'email': 'binomran@nandosqa.co.za', 'lastname': "Nando's", 'firstname': 'Bin Omran', 'password': '$2a$10$pqVhspIOiGSpOZa64bRDNOaR9s.NN9Q17YGZdNvV2/8AWNq3R4Dyi', 'created_at': DatetimeWithNanoseconds(2022, 3, 17, 12, 26, 47, 850000, tzinfo=<UTC>), 'created_by': <google.cloud.firestore_v1.document.DocumentReference object at 0x000002E08CFD5A90>, 'mobile_no': '+97430581991'}
0MlXkq4y5FbWpWT6mkcI: {'created_by': <google.cloud.firestore_v1.document.DocumentReference object at 0x000002E08D017400>, 'email': 'danielj2@onecart.co.za', 'created_at': DatetimeWithNanoseconds(2022, 10, 4, 12, 59, 

SyntaxError: 'return' outside function (4268899179.py, line 8)

In [19]:
data = pd.DataFrame.from_dict(docs_dict,orient='index')
data.head()

,created_by,email,created_at,firstname,password,lastname,mobile_no,user_ref,user_client_ref
0Gbd3UyDgGMsYKL2nj2H,<google.cloud.firestore_v1.document.DocumentRe...,vimals@nandocas.com,2021-07-06 10:15:09.947000+00:00,Vimal,$2a$10$LH3Bvph5uBIzX6w.faGHouq7jqMiq0EfKj1zCFG...,Singh,+27000000142,NaN,NaN
0IoJYDUaycxi0e5Fmozg,<google.cloud.firestore_v1.document.DocumentRe...,binomran@nandosqa.co.za,2022-03-17 12:26:47.850000+00:00,Bin Omran,$2a$10$pqVhspIOiGSpOZa64bRDNOaR9s.NN9Q17YGZdNv...,Nando's,+97430581991,NaN,NaN
0MlXkq4y5FbWpWT6mkcI,<google.cloud.firestore_v1.document.DocumentRe...,danielj2@onecart.co.za,2022-10-04 12:59:15.113000+00:00,Daniel,$2a$10$/OjLaHgtSIdUUmZQGX8xw.9wTnHhmHDAOiVR0.p...,Jankelow,+27795178925,NaN,NaN
0Nr53MEYd4o1nIRvDqMX,<google.cloud.firestore_v1.document.DocumentRe...,dom@notprod.com,2021-02-09 07:19:50.654000+00:00,Dom,$2a$10$0ILSudPYCETM3N.7N25MN.7MDXKRnkSRUeZXaEi...,Kafka,+27837453216,NaN,NaN
0e9zJpFNma5hr09QzjUP,<google.cloud.firestore_v1.document.DocumentRe...,mthatha@nandos.com,2021-07-09 08:05:19.419000+00:00,Mthatha,$2a$10$mMONOMkzh/xFg80VX0zy0uq3ms61ClvZ5ba168p...,Nando's,+27475319025,NaN,NaN


In [30]:
startdate = pd.to_datetime("2022-09-13").date()
enddate = pd.to_datetime("2022-10-27").date()

In [31]:
df = df[(df['date'] > startdate) & (df['date'] < enddate )]

In [25]:
df['date'] = df['created_at'].dt.date

In [34]:
df.to_csv('df_users.csv')

### ORDERS 

In [ ]:
#doc_ref = db.collection('clients').document('0C6fjhjLU6rnuoQyREOb').collection('orders').where(u"created_at", u">=", '9/13/22, 00:00 AM')

#orders_docs = [doc.to_dict() for doc in doc_ref.stream()]
#print(orders_docs)

#result = doc_ref.stream().to_dict()
#print(result)


"""collections = db.collection(u'clients').document('0C6fjhjLU6rnuoQyREOb').collection('orders').where(u"created_at", u">=", '9/13/22, 00:00 AM')
for collection in collections:
    for doc in collection.stream():
        print(f'{doc.id} => {doc.to_dict()}')"""

In [80]:
docs_dict = {}
docs = db.collection('clients').document('Ju2Ae08OZ83osvuIYkrs').collection('orders') 

for doc in docs.stream():
    print(f'{doc.id}: {doc.to_dict()}')
    docs_dict[doc.id] = doc.to_dict()
return docs_dict

0IvR41aGHqqOJ8ARDAEo: {'recreated': False, 'status': 'completed', 'order_no': '361242', 'delivery_time': DatetimeWithNanoseconds(2022, 9, 1, 7, 14, 12, 529000, tzinfo=<UTC>), 'assignable': True, 'branch': {'store_code': 'KKAICTestBranch', 'location': <google.cloud.firestore_v1._helpers.GeoPoint object at 0x000001727873F040>, 'address': '8 Evelyn Rd, Retreat, Cape Town, 7965, South Africa', 'name': 'Test Branch', 'id': 'i4N4X49BVl4HwvjyEPzg'}, 'customer': {'mobile_no': '+27619456236', 'name': 'Sopra'}, 'task_type': 'delivery', 'created_at': DatetimeWithNanoseconds(2022, 9, 1, 7, 14, 26, 782000, tzinfo=<UTC>), 'service_type': '', 'location': <google.cloud.firestore_v1._helpers.GeoPoint object at 0x0000017278E94F10>, 'delivery_flow': {'flows': [{'type': 'sog', 'completed': ['completed'], 'states': ['completed'], 'signatures': [{'completed': 'https://storage.googleapis.com/cb-prod-private-images/signature-images/Qbujm9yIdShNyRNOx6RT-Ju2Ae08OZ83osvuIYkrs-0IvR41aGHqqOJ8ARDAEo-signature.jpg?X

SyntaxError: 'return' outside function (1072813042.py, line 7)

In [81]:
data = pd.DataFrame.from_dict(docs_dict,orient='index')
data['client_name'] = "Kristen's Kick Ass Ice Cream"
data.head()

,recreated,status,order_no,delivery_time,assignable,branch,customer,task_type,created_at,service_type,...,abandon_flow,address,collection_time,clustered,origination,trip_id,history,type,alcohol,client_name
0IvR41aGHqqOJ8ARDAEo,False,completed,361242,2022-09-01 07:14:12.529000+00:00,True,"{'store_code': 'KKAICTestBranch', 'location': ...","{'mobile_no': '+27619456236', 'name': 'Sopra'}",delivery,2022-09-01 07:14:26.782000+00:00,,...,{'type': 'default'},"The Village Hub, Watsonia Lane, Scarborough, C...",2022-09-01 07:14:06.826000+00:00,False,manual,KTbCf3JX1tiV2jNVWSIy,"[{'status': 'pending', 'timestamp': 2022-09-01...",task,False,Kristen's Kick Ass Ice Cream
0Ok5NXkgeqDxUSMJbgYM,False,completed,26702,2022-08-12 09:30:28.731000+00:00,False,"{'address': 'Noordhoek Farm Village, Village L...","{'mobile_no': '+27836330834', 'name': 'Vicky S...",,2022-08-11 11:38:09.981000+00:00,,...,{'type': 'default'},"9 Firmount Road, Sea Point, Cape Town, South A...",2022-08-12 08:30:06.108000+00:00,False,manual,knEkfyu5WMFp1m5bIat7,"[{'status': 'pending', 'timestamp': 2022-08-11...",order,False,Kristen's Kick Ass Ice Cream
0SPtmsY1BuKOUAyGrs6b,False,completed,Constantia,2022-08-12 15:44:43.044000+00:00,False,{'location': <google.cloud.firestore_v1._helpe...,"{'name': 'Uitsig', 'mobile_no': '+27740762367'}",,2022-08-11 15:44:54.220000+00:00,,...,{'type': 'default'},"Kristen's Kick-Ass Ice Cream Constantia, Const...",2022-08-12 15:44:38.778000+00:00,False,manual,knEkfyu5WMFp1m5bIat7,"[{'status': 'pending', 'timestamp': 2022-08-11...",order,False,Kristen's Kick Ass Ice Cream
0bTCro5ZlqYAjYj4YYIc,False,completed,24726,2022-08-12 11:54:21.892000+00:00,False,{'location': <google.cloud.firestore_v1._helpe...,"{'mobile_no': '+27827831865', 'name': 'Derren ...",,2022-08-11 11:55:03.002000+00:00,,...,{'type': 'default'},"12 Turfontein Lane, Milnerton Ridge, Cape Town...",2022-08-12 11:54:15.946000+00:00,False,manual,knEkfyu5WMFp1m5bIat7,"[{'status': 'pending', 'timestamp': 2022-08-11...",order,False,Kristen's Kick Ass Ice Cream
1Xc4Nh7uXmmzJ9EutZqs,False,completed,1173 stock transfer,2022-08-19 07:12:42.068000+00:00,False,{'location': <google.cloud.firestore_v1._helpe...,"{'mobile_no': '+27847254284', 'name': 'Stellies'}",delivery,2022-08-19 07:12:58.817000+00:00,,...,{'type': 'default'},"Kristen's Kick-Ass Ice Cream Stellenbosch, Chu...",2022-08-19 07:12:55.066000+00:00,False,manual,aobYoEZrodMVzbCeYe8y,"[{'status': 'pending', 'timestamp': 2022-08-19...",task,False,Kristen's Kick Ass Ice Cream


In [82]:
data.shape

(121, 26)

In [83]:
df = data

In [84]:
df[['customer_mobile_no','customer_name']] = df.customer.apply(pd.Series)
df.head()

,recreated,status,order_no,delivery_time,assignable,branch,customer,task_type,created_at,service_type,...,collection_time,clustered,origination,trip_id,history,type,alcohol,client_name,customer_mobile_no,customer_name
0IvR41aGHqqOJ8ARDAEo,False,completed,361242,2022-09-01 07:14:12.529000+00:00,True,"{'store_code': 'KKAICTestBranch', 'location': ...","{'mobile_no': '+27619456236', 'name': 'Sopra'}",delivery,2022-09-01 07:14:26.782000+00:00,,...,2022-09-01 07:14:06.826000+00:00,False,manual,KTbCf3JX1tiV2jNVWSIy,"[{'status': 'pending', 'timestamp': 2022-09-01...",task,False,Kristen's Kick Ass Ice Cream,+27619456236,Sopra
0Ok5NXkgeqDxUSMJbgYM,False,completed,26702,2022-08-12 09:30:28.731000+00:00,False,"{'address': 'Noordhoek Farm Village, Village L...","{'mobile_no': '+27836330834', 'name': 'Vicky S...",,2022-08-11 11:38:09.981000+00:00,,...,2022-08-12 08:30:06.108000+00:00,False,manual,knEkfyu5WMFp1m5bIat7,"[{'status': 'pending', 'timestamp': 2022-08-11...",order,False,Kristen's Kick Ass Ice Cream,+27836330834,Vicky Sacks
0SPtmsY1BuKOUAyGrs6b,False,completed,Constantia,2022-08-12 15:44:43.044000+00:00,False,{'location': <google.cloud.firestore_v1._helpe...,"{'name': 'Uitsig', 'mobile_no': '+27740762367'}",,2022-08-11 15:44:54.220000+00:00,,...,2022-08-12 15:44:38.778000+00:00,False,manual,knEkfyu5WMFp1m5bIat7,"[{'status': 'pending', 'timestamp': 2022-08-11...",order,False,Kristen's Kick Ass Ice Cream,+27740762367,Uitsig
0bTCro5ZlqYAjYj4YYIc,False,completed,24726,2022-08-12 11:54:21.892000+00:00,False,{'location': <google.cloud.firestore_v1._helpe...,"{'mobile_no': '+27827831865', 'name': 'Derren ...",,2022-08-11 11:55:03.002000+00:00,,...,2022-08-12 11:54:15.946000+00:00,False,manual,knEkfyu5WMFp1m5bIat7,"[{'status': 'pending', 'timestamp': 2022-08-11...",order,False,Kristen's Kick Ass Ice Cream,+27827831865,Derren Page
1Xc4Nh7uXmmzJ9EutZqs,False,completed,1173 stock transfer,2022-08-19 07:12:42.068000+00:00,False,{'location': <google.cloud.firestore_v1._helpe...,"{'mobile_no': '+27847254284', 'name': 'Stellies'}",delivery,2022-08-19 07:12:58.817000+00:00,,...,2022-08-19 07:12:55.066000+00:00,False,manual,aobYoEZrodMVzbCeYe8y,"[{'status': 'pending', 'timestamp': 2022-08-19...",task,False,Kristen's Kick Ass Ice Cream,+27847254284,Stellies


In [87]:
# Kristen's Kick Ass Ice Cream
df7 = df[['client_name','order_no','trip_id','created_at','customer_name','customer_mobile_no']]

In [71]:
# Somerset Timbers
df6 = df[['client_name','order_no','trip_id','created_at','customer_name','customer_mobile_no']]

In [62]:
# Conerstone Pharmacy
df5 = df[['client_name','order_no','trip_id','created_at','customer_name','customer_mobile_no']]

In [55]:
# Busymed
df4 = df[['client_name','order_no','trip_id','created_at','customer_name','customer_mobile_no']]

In [46]:
# Nandos Qatar
df3 = df[['client_name','order_no','trip_id','created_at','customer_name','customer_mobile_no']]

In [33]:
#Geewiz
df2 = df[['client_name','order_no','trip_id','created_at','customer_name','customer_mobile_no']]

In [88]:
# append dataframes
df_final = pd.concat([df2, df3, df4, df5, df6, df7])
df_final.head()

,client_name,order_no,trip_id,created_at,customer_name,customer_mobile_no
07RqodhQuSAq0aBzZFRE,Geewiz,233711,dMqOJodEkiifIP2LJFMk,2022-08-12 07:05:09.901000+00:00,Chad Van Wyk,+27834537767
0dKdfwr5EHLUgC2gI37B,Geewiz,230885,QzG6WiyUsJjeFs19epsz,2022-08-05 06:59:28.609000+00:00,Kevin Lacey,+27615150421
281rXR8TwFxhdm2Bnpfw,Geewiz,230705,tJDeGjN6fJSC9vzFczJW,2022-08-03 09:45:13.901000+00:00,Stelio Hadjidemetriou,+27833086888
2LA2hW1sW3ibt4pW6o3j,Geewiz,232160,m5mLSRtroyOg7iPnCPmD,2022-08-04 07:08:47.064000+00:00,Abishek Tiwari,+27790208126
3GdnoKv439QWMaVLaPHs,Geewiz,230705,ds4gdeFi48ynf7vSQtl5,2022-08-03 08:02:26.246000+00:00,Stelio Hadjidemetriou,+2783308688


In [91]:
df_final['client_name'].unique()

array(['Geewiz', "Nando's Qatar", 'Busymed', ' Cornerstone Pharmacy',
       'Somerset Timbers', "Kristen's Kick Ass Ice Cream"], dtype=object)

In [90]:
# filter last 2 days
df_final2 = df_final.loc[(df_final['created_at'] >= '2022-09-13') & (df_final['created_at'] < '2022-09-16')]
df_final2.head()

,client_name,order_no,trip_id,created_at,customer_name,customer_mobile_no
6GlLgXYisEZXxCQvt8kY,Geewiz,239195,kMDYFd6ncDy49WG3PyYJ,2022-09-13 08:21:00.872000+00:00,Piero Arcangeli,+27835610100
IrNRMMtOo4gwPtHrplJm,Geewiz,239294,kMDYFd6ncDy49WG3PyYJ,2022-09-13 08:16:56.979000+00:00,Deon Schwabsky,+27729302112
QtMOuW6Edp40OHD1amsg,Geewiz,238165,kMDYFd6ncDy49WG3PyYJ,2022-09-13 08:11:38.893000+00:00,Rashid Vally,+27763646379
f30MtLRBPodhzoojpivE,Geewiz,239355,kMDYFd6ncDy49WG3PyYJ,2022-09-13 08:15:21.985000+00:00,Pieter Dorland,+27726017044
tZTDcUcuPb7kVe7RsNi7,Geewiz,238966,kMDYFd6ncDy49WG3PyYJ,2022-09-13 08:19:16.330000+00:00,Charles Loots,+27832746361


In [93]:
df_final2.to_csv('df_final2.csv')

In [6]:
docs_dict = {}
docs = db.collection('clients').document('6IB4FgVQZyQIG9T2xdEx').collection('orders') 

for doc in docs.stream():
    print(f'{doc.id}: {doc.to_dict()}')
    docs_dict[doc.id] = doc.to_dict()

In [ ]:
data = pd.DataFrame.from_dict(docs_dict,orient='index')
data['client_name'] = "Kristen's Kick Ass Ice Cream"
data.head()

### Drivers - Fleet Management

In [87]:
# For Nandos SA - zVznneXPn3Zaj75n5Q3N
categoryDocRef = db.collection('clients').document('zVznneXPn3Zaj75n5Q3N')

file = db.collection('drivers').where('client', '==', categoryDocRef)

drivers_docs = [doc.to_dict() for doc in file.stream()]
print(drivers_docs)

[{'name': 'Jeremiah Honwane', 'on_active_trip': False, 'created_by': <google.cloud.firestore_v1.document.DocumentReference object at 0x00000171E4073550>, 'vehicle_type': 'bike', 'employee_code': 'Eldorado 05', 'idle_since': DatetimeWithNanoseconds(2022, 6, 3, 0, 0, 24, 648000, tzinfo=<UTC>), 'client': <google.cloud.firestore_v1.document.DocumentReference object at 0x00000171E4073730>, 'last_trip_rejection': DatetimeWithNanoseconds(2022, 5, 15, 14, 35, 18, 556000, tzinfo=<UTC>), 'delivery_permissions': [{'promisor_id': 'zVznneXPn3Zaj75n5Q3N', 'hub_id': 'GNwtEW1ukowCbgtxIQF1'}], 'available': False, 'mobile_no': '+27839553918 ', 'push_token': 'e_zGh0JbRM2YxS0CtrE3Gx:APA91bFhlF9EGCOO1D8tswS6MkgffOYKA-BUwzAWoyYXMLzDpAucOuDMhXjUHQPlhn82hhKYY5ivtdZ_OJk03hezMnccMefeuPdB4mdpqEtSl5mnI0HrADCOK8xqoNEd1YnmYU6Roh3H', 'location': {'bearingAccuracy': 30.0, 'speedAccuracy': 0.0, 'accuracy': 30.0, 'verticalAccuracy': 5, 'latitude': -26.2917338, 'longitude': 27.9067851, 'bearing': 0, 'provider': '', 'tim

In [90]:
drivers_df = pd.DataFrame.from_dict(orders_docs)
drivers_df.head()

,name,on_active_trip,created_by,vehicle_type,employee_code,idle_since,client,last_trip_rejection,delivery_permissions,available,...,created_at,password,lunch,email,blocked,sos,active_trip,image,primary_employer,scheduled_lunch
0,Jeremiah Honwane,False,<google.cloud.firestore_v1.document.DocumentRe...,bike,Eldorado 05,2022-06-03 00:00:24.648000+00:00,<google.cloud.firestore_v1.document.DocumentRe...,2022-05-15 14:35:18.556000+00:00,"[{'promisor_id': 'zVznneXPn3Zaj75n5Q3N', 'hub_...",False,...,2021-06-13 13:38:44.839000+00:00,$2a$10$FKmypciLURtHmqQj4rz5veAyVRtaynx7BHFAU.c...,False,5@eldo.com,False,False,NaN,NaN,NaN,NaN
1,Machona,False,<google.cloud.firestore_v1.document.DocumentRe...,bike,Eldoraigne 08,2021-10-04 00:00:51.052000+00:00,<google.cloud.firestore_v1.document.DocumentRe...,2021-10-03 16:08:07.360000+00:00,[],False,...,2021-09-14 10:45:07.834000+00:00,$2a$10$Q/in1/WlUBDeu2llRUsJ7OnsgtQNSHqyI8VdzUA...,False,8@eldor.com,True,NaN,NaN,NaN,NaN,NaN
2,Junite Mashele,True,<google.cloud.firestore_v1.document.DocumentRe...,e-bike,Polokwane 05,2022-09-29 10:55:48.317000+00:00,<google.cloud.firestore_v1.document.DocumentRe...,2022-09-24 10:55:40.333000+00:00,"[{'promisor_id': 'zVznneXPn3Zaj75n5Q3N', 'hub_...",True,...,2021-08-20 12:43:36.369000+00:00,$2a$10$G1ahH0GfsRzCrjjFzGecH.3edaqBaKAbmGlyLTQ...,False,5@polo.com,False,False,"{'client_id': 'zVznneXPn3Zaj75n5Q3N', 'trip_id...",NaN,NaN,NaN
3,Gcina Nyawose,False,<google.cloud.firestore_v1.document.DocumentRe...,bike,Ballito01,2022-04-10 00:00:09.706000+00:00,<google.cloud.firestore_v1.document.DocumentRe...,2021-12-30 14:24:29.661000+00:00,"[{'hub_id': 'PSFH1asmyBZC7P2e8x82', 'promisor_...",False,...,2021-05-17 14:17:34.669000+00:00,$2a$10$PkxEvuRrW5jwryP5JjfPZeCzJRN2wBzaoVM4aXp...,False,1@ballito.com,False,NaN,NaN,NaN,NaN,NaN
4,Rashid Justin,False,<google.cloud.firestore_v1.document.DocumentRe...,bike,Trade Route 02,2022-09-29 12:13:30.598000+00:00,<google.cloud.firestore_v1.document.DocumentRe...,NaT,"[{'hub_id': 'P2lTm0pkX06tKTgaJfze', 'promisor_...",True,...,2022-04-25 07:21:14.053000+00:00,$2a$10$MMHoichSjJqoKwzohgkf8e5KnjVqMwF2wf/TaTo...,False,2@trade.com,False,NaN,NaN,NaN,NaN,NaN


In [92]:
drivers_df['delivery_permissions'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 435 entries, 0 to 434
Series name: delivery_permissions
Non-Null Count  Dtype 
--------------  ----- 
431 non-null    object
dtypes: object(1)
memory usage: 3.5+ KB


In [91]:
drivers_df.to_csv('drivers_df.csv')

## Client Admin Users for a single client

In [11]:
docs_dict = {}
docs = db.collection('clients').document('zVznneXPn3Zaj75n5Q3N').collection('users') 

for doc in docs.stream():
    print(f'{doc.id}: {doc.to_dict()}')
    docs_dict[doc.id] = doc.to_dict()
return docs_dict

0Kdtg4GdEmRYjbQfYkEg: {'created_at': DatetimeWithNanoseconds(2021, 7, 1, 13, 38, 40, 190000, tzinfo=<UTC>), 'mobile_no': '+27126540276', 'user_ref': <google.cloud.firestore_v1.document.DocumentReference object at 0x000001B28F730BB0>, 'firstname': 'Eldoraigne', 'lastname': "Nando's", 'created_by': <google.cloud.firestore_v1.document.DocumentReference object at 0x000001B28F730FD0>, 'password': '$2a$10$KOgo6jqOhcvfFoii66GR2eWXSZuPgwP/h7u6CJk6O5Bt0qiUfHCti', 'permissions': {'fleet': False, 'dos': {'regions': [<google.cloud.firestore_v1.document.DocumentReference object at 0x000001B28F730670>], 'hubs': [<google.cloud.firestore_v1.document.DocumentReference object at 0x000001B28F730580>]}, 'administrator': False}, 'user_client_ref': <google.cloud.firestore_v1.document.DocumentReference object at 0x000001B28F730970>, 'email': 'eldoraigne@nandocas.com'}
0irvuS9de8jxvmFR0ioA: {'mobile_no': '+27325338966', 'user_ref': <google.cloud.firestore_v1.document.DocumentReference object at 0x000001B28F73

SyntaxError: 'return' outside function (545224293.py, line 7)

In [12]:
data = pd.DataFrame.from_dict(docs_dict,orient='index')
data.head()

,created_at,mobile_no,user_ref,firstname,lastname,created_by,password,permissions,user_client_ref,email,image
0Kdtg4GdEmRYjbQfYkEg,2021-07-01 13:38:40.190000+00:00,+27126540276,<google.cloud.firestore_v1.document.DocumentRe...,Eldoraigne,Nando's,<google.cloud.firestore_v1.document.DocumentRe...,$2a$10$KOgo6jqOhcvfFoii66GR2eWXSZuPgwP/h7u6CJk...,"{'fleet': False, 'dos': {'regions': [<google.c...",<google.cloud.firestore_v1.document.DocumentRe...,eldoraigne@nandocas.com,NaN
0irvuS9de8jxvmFR0ioA,2021-07-09 09:20:21.304000+00:00,+27325338966,<google.cloud.firestore_v1.document.DocumentRe...,Verulam,Nando's,<google.cloud.firestore_v1.document.DocumentRe...,$2a$10$rLkcOcuisBArwmCEPSkAxuk1gq6KwQ4COkLIiK2...,"{'fleet': False, 'administrator': False, 'dos'...",<google.cloud.firestore_v1.document.DocumentRe...,verulam@nandos.com,NaN
16sUkRuoOWeQYTuSQtgw,2021-05-27 07:49:13.967000+00:00,+27000000003,<google.cloud.firestore_v1.document.DocumentRe...,Alicia,van der Merwe,<google.cloud.firestore_v1.document.DocumentRe...,$2a$10$kwd3C/oIKvVFw/dCFHwbYOH6J0Ddt/2WWwT0hTc...,"{'fleet': True, 'administrator': False, 'dos':...",<google.cloud.firestore_v1.document.DocumentRe...,alicias@nandos.com,NaN
1FQ4yPAcnAVoNxvI6yVc,2021-05-31 06:04:09.253000+00:00,+27825507456,<google.cloud.firestore_v1.document.DocumentRe...,Anandi,Rautenbach,<google.cloud.firestore_v1.document.DocumentRe...,$2a$10$B7cE3FOqiWYCot3yrtjS6e4sMb6PS3RLlLpdxRl...,"{'fleet': True, 'administrator': True, 'dos': ...",<google.cloud.firestore_v1.document.DocumentRe...,anandi@nandos.com,NaN
21iXY2iqEEVxCgV0y0ex,2021-07-09 09:25:00.376000+00:00,+27217030022,<google.cloud.firestore_v1.document.DocumentRe...,Ottery,Nando's,<google.cloud.firestore_v1.document.DocumentRe...,$2a$10$ekVbJgfTVKSvcDY5d7kgouHfS.e4qzJrRVVgaFG...,"{'administrator': False, 'fleet': False, 'dos'...",<google.cloud.firestore_v1.document.DocumentRe...,ottery@nandocas.com,NaN


In [15]:
data.to_csv('nandos_users.csv')

## TRIPS

In [4]:
docs_dict = {}
docs = db.collection('clients').document('B0l6eGBH0UB7wOhWzZ9J').collection('trips') 

for doc in docs.stream():
    #print(f'{doc.id}: {doc.to_dict()}')
    docs_dict[doc.id] = doc.to_dict()
#return docs_dict

AttributeError: '_UnaryStreamMultiCallable' object has no attribute '_retry'

In [25]:
# pull documents/trips in batches
batch_size = 50
docs_dict = {}
docs = db.collection('clients').document('B0l6eGBH0UB7wOhWzZ9J').collection('trips') 

next_doc = docs.limit(1).get()

while next_doc:
    for doc in next_doc:
        docs_dict[doc.id] = doc.to_dict()
    next_doc = docs.start_after(doc).limit(batch_size).get()

In [26]:
data = pd.DataFrame.from_dict(docs_dict,orient='index')
data.head()

,distance,branch,automatic_assignment,orders,history,activity,status,duration,expired,dispatch,driver,created_at,wallet,created_by,compute_route,task
001sOok6Jkurqujwpdyv,3.15,<google.cloud.firestore_v1.document.DocumentRe...,True,[df3aKo1ip2MJsqkYiTi8],"[{'status': 'pending', 'timestamp': 2022-11-29...","[{'driver_id': 'iFSapTeYtixyrns5sMfE', 'timest...",completed,7 mins,False,sequential,<google.cloud.firestore_v1.document.DocumentRe...,2022-11-29 09:44:47.192000+00:00,"{'tip_amount': 10, 'cost': 50}",NaN,NaN,NaN
002TAdHy9ORk8Ln9QqtI,4.78,<google.cloud.firestore_v1.document.DocumentRe...,True,[BnFMTzfFQrhJBAjkTm2G],[{'timestamp': 2022-12-16 14:27:14.204000+00:0...,"[{'driver_id': 'vNYIDK7JryQjjRlgrxSW', 'timest...",completed,10 mins,False,sequential,<google.cloud.firestore_v1.document.DocumentRe...,2022-12-16 14:27:14.211000+00:00,"{'cost': 70, 'tip_amount': 30}",NaN,NaN,NaN
003H6EYh2ZuXmyp4FfCc,5.75,<google.cloud.firestore_v1.document.DocumentRe...,True,[AW7MGepW2TgM0qokLfOa],"[{'status': 'pending', 'timestamp': 2022-11-25...",[{'timestamp': 2022-11-25 15:08:06.949000+00:0...,completed,12 mins,False,sequential,<google.cloud.firestore_v1.document.DocumentRe...,2022-11-25 15:08:02.257000+00:00,"{'tip_amount': 10, 'cost': 56}",NaN,NaN,NaN
004fnVvcSaNkmDnIhwPp,3.42,<google.cloud.firestore_v1.document.DocumentRe...,True,[WFJBJcRkzCbkthMrkywz],"[{'status': 'pending', 'timestamp': 2022-12-12...","[{'driver_id': '3IP9UWrtIV2801szmUis', 'timest...",completed,8 mins,False,sequential,<google.cloud.firestore_v1.document.DocumentRe...,2022-12-12 08:46:14.193000+00:00,"{'cost': 50, 'tip_amount': 10}",NaN,NaN,NaN
008Q37lkEREA3MqZ5amV,2.33,<google.cloud.firestore_v1.document.DocumentRe...,True,[dS0lnxdJraemoBaNrrIv],"[{'status': 'pending', 'timestamp': 2022-12-04...",[{'timestamp': 2022-12-04 09:15:13.774000+00:0...,completed,7 mins,False,sequential,<google.cloud.firestore_v1.document.DocumentRe...,2022-12-04 09:14:34.178000+00:00,"{'cost': 70, 'tip_amount': 20}",NaN,NaN,NaN


In [27]:
data['trip_id'] = data.index

In [28]:
data.head()

,distance,branch,automatic_assignment,orders,history,activity,status,duration,expired,dispatch,driver,created_at,wallet,created_by,compute_route,task,trip_id
001sOok6Jkurqujwpdyv,3.15,<google.cloud.firestore_v1.document.DocumentRe...,True,[df3aKo1ip2MJsqkYiTi8],"[{'status': 'pending', 'timestamp': 2022-11-29...","[{'driver_id': 'iFSapTeYtixyrns5sMfE', 'timest...",completed,7 mins,False,sequential,<google.cloud.firestore_v1.document.DocumentRe...,2022-11-29 09:44:47.192000+00:00,"{'tip_amount': 10, 'cost': 50}",NaN,NaN,NaN,001sOok6Jkurqujwpdyv
002TAdHy9ORk8Ln9QqtI,4.78,<google.cloud.firestore_v1.document.DocumentRe...,True,[BnFMTzfFQrhJBAjkTm2G],[{'timestamp': 2022-12-16 14:27:14.204000+00:0...,"[{'driver_id': 'vNYIDK7JryQjjRlgrxSW', 'timest...",completed,10 mins,False,sequential,<google.cloud.firestore_v1.document.DocumentRe...,2022-12-16 14:27:14.211000+00:00,"{'cost': 70, 'tip_amount': 30}",NaN,NaN,NaN,002TAdHy9ORk8Ln9QqtI
003H6EYh2ZuXmyp4FfCc,5.75,<google.cloud.firestore_v1.document.DocumentRe...,True,[AW7MGepW2TgM0qokLfOa],"[{'status': 'pending', 'timestamp': 2022-11-25...",[{'timestamp': 2022-11-25 15:08:06.949000+00:0...,completed,12 mins,False,sequential,<google.cloud.firestore_v1.document.DocumentRe...,2022-11-25 15:08:02.257000+00:00,"{'tip_amount': 10, 'cost': 56}",NaN,NaN,NaN,003H6EYh2ZuXmyp4FfCc
004fnVvcSaNkmDnIhwPp,3.42,<google.cloud.firestore_v1.document.DocumentRe...,True,[WFJBJcRkzCbkthMrkywz],"[{'status': 'pending', 'timestamp': 2022-12-12...","[{'driver_id': '3IP9UWrtIV2801szmUis', 'timest...",completed,8 mins,False,sequential,<google.cloud.firestore_v1.document.DocumentRe...,2022-12-12 08:46:14.193000+00:00,"{'cost': 50, 'tip_amount': 10}",NaN,NaN,NaN,004fnVvcSaNkmDnIhwPp
008Q37lkEREA3MqZ5amV,2.33,<google.cloud.firestore_v1.document.DocumentRe...,True,[dS0lnxdJraemoBaNrrIv],"[{'status': 'pending', 'timestamp': 2022-12-04...",[{'timestamp': 2022-12-04 09:15:13.774000+00:0...,completed,7 mins,False,sequential,<google.cloud.firestore_v1.document.DocumentRe...,2022-12-04 09:14:34.178000+00:00,"{'cost': 70, 'tip_amount': 20}",NaN,NaN,NaN,008Q37lkEREA3MqZ5amV


In [31]:
credentials = Credentials.from_service_account_file('C:/Users/leemn/Documents/Loop Service Account/cb-prod-297913-623cea238b9a.json')

query_orders = '''
SELECT DISTINCT client_id,order_id, order_no, order_status, created_at, trip_id, 
hub_id, branch_id, branch_name, driver.id AS driver_id, driver.name AS driver_name, 
driver.employee_code AS employee_code, payment_setting, driver_earnings, driver_tips, distance 
FROM `cb-prod-297913.cb_prod_analytics.cb_driver_payment_wallet_2` wallet_2 
WHERE client_id IN ('B0l6eGBH0UB7wOhWzZ9J') 
GROUP BY 1,2,3,4,5,6,7,8,9,10,10,11,12,13,14,15,16 
UNION ALL SELECT DISTINCT client_id,order_id, order_no, order_status, created_at, trip_id, hub_id,
branch_id, branch_name, d.id AS driver_id, d.name AS driver_name, d.employee_code AS employee_code, payment_setting, 
driver_earnings, driver_tips, distance FROM `cb-prod-297913.cb_prod_analytics.cb_driver_payment_wallet` wallet,
UNNEST(wallet.driver) AS d WHERE client_id IN ('B0l6eGBH0UB7wOhWzZ9J') 
GROUP BY 1,2,3,4,5,6,7,8,9,10,10,11,12,13,14,15,16
'''
df_orders = pd.read_gbq(query_orders, project_id='cb-prod-297913', dialect='standard', credentials=credentials,progress_bar_type='tqdm_notebook')

Downloading:   0%|          | 0/133290 [00:00<?, ?rows/s]

In [32]:
df_final_onecart = pd.merge(data, df_orders, how='left', on=['trip_id'])
df_final_onecart.head()

,distance_x,branch,automatic_assignment,orders,history,activity,status,duration,expired,dispatch,...,hub_id,branch_id,branch_name,driver_id,driver_name,employee_code,payment_setting,driver_earnings,driver_tips,distance_y
0,3.15,<google.cloud.firestore_v1.document.DocumentRe...,True,[df3aKo1ip2MJsqkYiTi8],"[{'status': 'pending', 'timestamp': 2022-11-29...","[{'driver_id': 'iFSapTeYtixyrns5sMfE', 'timest...",completed,7 mins,False,sequential,...,OvwjsJGfldeb0DNp22zs,26OpfmnOtNk6IUsbmCgB,Woolworths Vincent Park Shopping Mall,iFSapTeYtixyrns5sMfE,Macdonald Masvosva,W-1040,Woolworths Drivers,50.0,10.0,3.15
1,4.78,<google.cloud.firestore_v1.document.DocumentRe...,True,[BnFMTzfFQrhJBAjkTm2G],[{'timestamp': 2022-12-16 14:27:14.204000+00:0...,"[{'driver_id': 'vNYIDK7JryQjjRlgrxSW', 'timest...",completed,10 mins,False,sequential,...,B2kxdWPWe3Lrl8VBO47H,CqHlcD5DaOnb7QPz0NW6,Woolworths CapeGate Shopping Centre,vNYIDK7JryQjjRlgrxSW,Florent Nkumingo,W-1012,Woolworths Drivers,70.0,30.0,4.78
2,5.75,<google.cloud.firestore_v1.document.DocumentRe...,True,[AW7MGepW2TgM0qokLfOa],"[{'status': 'pending', 'timestamp': 2022-11-25...",[{'timestamp': 2022-11-25 15:08:06.949000+00:0...,completed,12 mins,False,sequential,...,0aYB8yzjrYdDpbZoukUB,zEmLre6WYC5ipb8k3DsC,Woolworths Nicolway,K4c1hOruwF3MdjCc0YDW,Moise Kabongo,W-1113,Woolworths Drivers,56.0,10.0,5.75
3,3.42,<google.cloud.firestore_v1.document.DocumentRe...,True,[WFJBJcRkzCbkthMrkywz],"[{'status': 'pending', 'timestamp': 2022-12-12...","[{'driver_id': '3IP9UWrtIV2801szmUis', 'timest...",completed,8 mins,False,sequential,...,hFYcwx0kGRzxlwaeTLch,FskgkdM4jLbM34LAfv1m,Woolworths Benmore Shopping Centre,3IP9UWrtIV2801szmUis,Gift Tembo,WooliesDash46,Woolworths Drivers,50.0,10.0,3.42
4,2.33,<google.cloud.firestore_v1.document.DocumentRe...,True,[dS0lnxdJraemoBaNrrIv],"[{'status': 'pending', 'timestamp': 2022-12-04...",[{'timestamp': 2022-12-04 09:15:13.774000+00:0...,completed,7 mins,False,sequential,...,hFYcwx0kGRzxlwaeTLch,FskgkdM4jLbM34LAfv1m,Woolworths Benmore Shopping Centre,a6sXYhlFL0Crfrmy6mFD,Kalonji kangudia vainquer WOOLIES,WooliesDash456,Woolworths Drivers,70.0,20.0,2.33


In [34]:
df_final_onecart['trip_id'].nunique(), data['trip_id'].nunique(), df_orders['trip_id'].nunique()

(139407, 139407, 132708)

In [35]:
df_final_onecart.to_csv('df_final_onecart233.csv')

In [36]:
# trips in firestore but not in df_orders (from BQ)
ids_in_df2 = data.trip_id.unique()
not_found_ids = df_orders[~df_orders['trip_id'].isin(ids_in_df2 )]

In [40]:
not_found_ids.shape

(5, 16)

In [42]:
not_found_ids.head(20)

,client_id,order_id,order_no,order_status,created_at,trip_id,hub_id,branch_id,branch_name,driver_id,driver_name,employee_code,payment_setting,driver_earnings,driver_tips,distance
30430,B0l6eGBH0UB7wOhWzZ9J,FZd8OlRbvbFfUoYeLIvg,3278941,delivered,2022-12-19 15:05:48+00:00,APgKhBNOLH9jCOAlX0GD,JGMKUm4Wi20IPN3ejX5E,vWWDUD57pYf9GQgU9QaM,Woolworths Woodmead Retail Park,tXzjJI4xid9nNeOLWpNX,Vuyisile Ndebele,WD987,BikeSolve OneCart,46.0,10.0,5.10
61813,B0l6eGBH0UB7wOhWzZ9J,M6peUHdBIJSh17SB6Ke8,3278950,delivered,2022-12-19 15:10:32+00:00,35O1R7xC2wOnCaqk43ER,Y7DwuFGYfZZ3bmFvUb4D,5Ceh7DUDum4NWr35XH4t,Woolworths Gordons Bay,iKlcwOIIX5tn5okcqHAn,Gift Kayeni,WooliesDash07,Woolworths Drivers,56.0,10.0,5.94
101009,B0l6eGBH0UB7wOhWzZ9J,0VcP6Oha2khYXWWIwXqw,3278901,delivered,2022-12-19 14:58:56+00:00,1GeL6ws8pP5eijAD0hyk,l2d7ojRLTrNlbfpEkCov,S3UJbtExu2eitATFa3td,Woolworths Jean Crossing,cYsQUzwNvbccJUtyRYHL,Clever Nduruza,wdash-4082,Lularides Woolies Dash Drivers,30.0,0.0,2.14
118901,B0l6eGBH0UB7wOhWzZ9J,CQsYTXVhtw1POfP0h07X,3278963,delivered,2022-12-19 15:10:33+00:00,Kz1AGHgRFmyfxx6KMoSE,tcnSNuNMUk32rOc6HRvN,aVE33UP22H8ggMtzTGLd,Woolworths Lillies Quarters Shopping Mall,jt2xSrh4IJX2ZZZeYQ9k,Nosipho Ntombela,wdash-4045,Woolworths Drivers,60.0,20.0,3.99
126035,B0l6eGBH0UB7wOhWzZ9J,KRwEdJtuwMcNI6YlSU1r,3278897,delivered,2022-12-19 15:04:56+00:00,4X5wLTy5Tgg1PKFthpOK,umTPnwARdmoc0mSJFDvp,oDucQkBt9fKlomiClNzM,Woolworths Park & Shop,3o4mbibTHpo0UfHmDgrb,Bigson Maphosa,WooliesDash92,Woolworths Drivers,50.0,10.0,2.80
